In [1]:
import jax
jax.config.update("jax_platform_name", "cpu")
jax.config.update("jax_enable_x64", True)
import numpy as np
np.set_printoptions(precision=5, linewidth=150)
import spax
from datetime import datetime
from sklearn.decomposition import PCA

# Principle Component Analysis (PCA)
Let X be $n \times m$ matrix, where $n$ represents dimensionality of the data and $m$ number of samples.
To prepare data for the PCA, it should be centered `X -= X.mean(axis = 1)`, and possibly whitened `X /= X.std(axis = 1)`. Whitening is done to scale dimensions to unit variance. This is specifically useful if data dimensions are in different units or represent different observables. PCA then consists of finding a rotation matrix which makes covariance of the data diagonal. Specifically, PCA of order $N$ does this rotation for $N$ largest eigenvalues of the covariance matrix.

## Singular Value Decomposition (SVD)
Every non-quadratic matrix can be decomposed as $$X = U \, S \, V^T \, ,$$ where $U$ is $n \times m$ matrix with orthonormal columns ($U^T \, U = I$), $V$ is $m \times m$ orthonormal matrix ($V^T \, V = I$) and $S$ a diagonal $m\times m$ matrix of singular values. From such decomposition one can write
\begin{align}
X \, X^T &= U \, S^2 \, U^T  \, , \\
X^T X &= V \, S^2 \, V^T \, .
\end{align}
Both $X \, X^T$ and $X^T X$ have the same eigenvalues, with larger of the two having the rest equal to $0$.

## Case #1: $n \le m$
Covariance of the data can be written as $$ C = \frac{1}{m-1} X \, X^T = U \, \frac{S^2}{m-1} \, U^T \, .$$ Therefore, by solving an eigenvalue problem for $C$, we can find $U$. By picking $N$ eigenvectors in the directions of the largest eigenvalues, we construct $\widetilde{U}$ used for PCA.

For some matrix $X_0$ of size $n \times m_0$, PCA is simply $$ Y_0 = \widetilde{U}^T X_0$$.

In [2]:
N_dim, N_samples = (16, 10**5)
pca = spax.PCA_m(5, devices = jax.devices("gpu"))
data = np.random.normal(0, 1, size = (N_dim, N_samples)) * np.sqrt(np.arange(1, N_dim + 1))[:, np.newaxis]
tic = datetime.now()
pca.fit(data, batch_size = N_dim // 2) # N_dim % (N_devices * batch_size) == 0
print("DURATION:", datetime.now() - tic)
sampled_data = pca.sample(N_samples)
print(np.std(pca.transform(data), axis = 1)**2) # should be [8, 7, 6, 5, 4]
print(np.std(pca.transform(sampled_data), axis = 1)**2) #should be the same
print(pca.λ ** 2) # should be the same
print(np.round(pca.U.T, 1)) # should be a +-unit matrix on last 5 dimensions

DURATION: 0:00:03.737178
[16.01832 14.94243 14.06906 13.01047 11.98974]
[16.06469 14.86496 14.11905 13.01072 12.08849]
[16.01968 14.94319 14.06939 13.01245 11.99197]
[[-0.  -0.   0.   0.  -0.   0.   0.   0.  -0.   0.   0.   0.  -0.   0.  -0.1  1. ]
 [-0.  -0.  -0.  -0.   0.   0.   0.  -0.   0.   0.  -0.   0.   0.  -0.   1.   0.1]
 [-0.  -0.   0.  -0.   0.  -0.  -0.   0.   0.  -0.   0.  -0.  -0.   1.   0.  -0. ]
 [-0.   0.  -0.  -0.   0.   0.   0.  -0.   0.  -0.   0.  -0.   1.   0.  -0.   0. ]
 [ 0.  -0.  -0.   0.   0.   0.  -0.  -0.  -0.  -0.  -0.   1.   0.   0.  -0.  -0. ]]


In [3]:
#testing the result with scikit-learn
pca_sk = PCA(n_components = 5)
tic = datetime.now()
pca_sk.fit(data.T)
print("DURATION:", datetime.now() - tic)
print(np.std(pca_sk.transform(data.T), axis = 0)**2)
print(np.std(pca_sk.transform(sampled_data.T), axis = 0)**2)
print(pca_sk.singular_values_**2 / (N_samples - 1))
print(np.round(pca_sk.components_, 1))

DURATION: 0:00:00.241689
[16.01932 14.94344 14.06947 13.01223 11.99135]
[16.06306 14.86302 14.1183  13.00811 12.08559]
[16.01948 14.94359 14.06961 13.01236 11.99147]
[[ 0.  -0.  -0.   0.   0.  -0.   0.  -0.   0.  -0.  -0.  -0.   0.  -0.   0.1 -1. ]
 [-0.   0.   0.   0.  -0.   0.  -0.  -0.  -0.  -0.   0.  -0.  -0.   0.  -1.  -0.1]
 [ 0.  -0.  -0.  -0.  -0.   0.  -0.  -0.  -0.   0.  -0.   0.   0.  -1.  -0.   0. ]
 [ 0.   0.  -0.  -0.  -0.  -0.  -0.   0.  -0.   0.  -0.   0.  -1.  -0.   0.  -0. ]
 [-0.   0.  -0.  -0.   0.  -0.   0.  -0.   0.   0.   0.  -1.  -0.  -0.   0.   0. ]]


## Case #2 $n \ge m$
In this case, it is better to write: $$D = \frac{1}{n} X^T \, X = V \, \frac{S^2}{n} \, V^T \, .$$
Solving eigenvector problem for $D$ gives us $V$ and $S$. Then, rotation matrix can be computed as $$ U = X \, V \, S^{-1} \, .$$ The rest is the same as in previous case.

In [4]:
N_dim, N_samples = (10**5, 16)
pca = spax.PCA_m(5, devices = jax.devices("gpu"))
data = np.random.normal(0, 1, size = (N_dim, N_samples)) * np.sqrt(np.arange(1, N_dim + 1))[:, np.newaxis]
tic = datetime.now()
pca.fit(data, batch_size = N_dim // 2, centering_data = "CPU") # N_dim % (N_devices * batch_size) == 0
print("DURATION:", datetime.now() - tic)
sampled_data = pca.sample(10**4)
print(np.std(pca.transform(data), axis = 1)**2)
print(np.std(pca.transform(sampled_data), axis = 1)**2)
print(pca.λ ** 2)
print(pca.U.T)

DURATION: 0:00:01.733989
[3.19629e+08 3.17828e+08 3.16138e+08 3.14550e+08 3.13497e+08]
[3.48978e+08 3.31489e+08 3.34646e+08 3.28235e+08 3.25079e+08]
[3.40938e+08 3.39016e+08 3.37214e+08 3.35520e+08 3.34397e+08]
[[ 2.39659e-06  4.00095e-05  4.84926e-05 ... -8.48269e-03 -2.74944e-03  4.68330e-04]
 [-1.68817e-05  2.94139e-05 -4.72969e-05 ...  6.10393e-04 -3.46886e-03  5.52197e-03]
 [ 8.89183e-06 -2.25058e-05 -2.10015e-06 ...  6.15842e-03  3.02637e-03 -2.04376e-03]
 [ 2.37999e-05 -1.11095e-05  4.74709e-05 ... -3.62896e-03  3.55384e-03  5.13295e-03]
 [ 1.25039e-05 -1.41444e-06 -7.36381e-06 ... -5.48890e-03 -5.40354e-03  5.72205e-03]]


In [5]:
pca_sk = PCA(n_components = 5)
tic = datetime.now()
pca_sk.fit(data.T)
print("DURATION:", datetime.now() - tic)
print(np.std(pca_sk.transform(data.T), axis = 0)**2)
print(np.std(pca_sk.transform(sampled_data.T), axis = 0)**2)
print(pca_sk.singular_values_**2 / (N_samples - 1))
print(pca_sk.components_)

DURATION: 0:00:00.172175
[3.19629e+08 3.17828e+08 3.16138e+08 3.14550e+08 3.13497e+08]
[3.48978e+08 3.31489e+08 3.34646e+08 3.28235e+08 3.25079e+08]
[3.40937e+08 3.39017e+08 3.37214e+08 3.35520e+08 3.34397e+08]
[[ 2.39657e-06  4.00093e-05  4.84930e-05 ... -8.48271e-03 -2.74947e-03  4.68297e-04]
 [-1.68817e-05  2.94142e-05 -4.72959e-05 ...  6.10387e-04 -3.46887e-03  5.52199e-03]
 [ 8.89109e-06 -2.25052e-05 -2.10035e-06 ...  6.15854e-03  3.02638e-03 -2.04395e-03]
 [ 2.38013e-05 -1.11104e-05  4.74696e-05 ... -3.62901e-03  3.55413e-03  5.13322e-03]
 [ 1.25033e-05 -1.41382e-06 -7.36406e-06 ... -5.48887e-03 -5.40369e-03  5.72208e-03]]


## Test save + load

In [6]:
pca.save("test.hdf5")
pca_new = spax.PCA_m(5, devices = jax.devices("gpu"))
pca_new.load("test.hdf5")

x0 = np.arange(100000)[:, np.newaxis]
print(pca.transform(x0).flatten())
print(pca_new.transform(x0).flatten())
print(pca_sk.transform(x0.T).flatten())

[-53052.11  -32735.893 -74252.41   47275.414 -21955.748]
[-53052.11  -32735.893 -74252.41   47275.414 -21955.748]
[-53051.85338 -32736.08236 -74254.23987  47277.47574 -21957.7989 ]
